# 480. Sliding Window Median (Hard)

<div><p>Median is the middle value in an ordered integer list. If the size of the list is even, there is no middle value. So the median is the mean of the two middle value.</p>
Examples:

<p><code>[2,3,4]</code> , the median is <code>3</code></p>

<p><code>[2,3]</code>, the median is <code>(2 + 3) / 2 = 2.5</code></p>

<p>Given an array <i>nums</i>, there is a sliding window of size <i>k</i> which is moving from the very left of the array to the very right. You can only see the <i>k</i> numbers in the window. Each time the sliding window moves right by one position. Your job is to output the median array for each window in the original array.</p>

<p>For example,<br>
Given <i>nums</i> = <code>[1,3,-1,-3,5,3,6,7]</code>, and <i>k</i> = 3.</p>

<pre>Window position                Median
---------------               -----
[1  3  -1] -3  5  3  6  7       1
 1 [3  -1  -3] 5  3  6  7       -1
 1  3 [-1  -3  5] 3  6  7       -1
 1  3  -1 [-3  5  3] 6  7       3
 1  3  -1  -3 [5  3  6] 7       5
 1  3  -1  -3  5 [3  6  7]      6
</pre>

<p>Therefore, return the median sliding window as <code>[1,-1,-1,3,5,6]</code>.</p>

<p><b>Note: </b><br>
You may assume <code>k</code> is always valid, ie: <code>k</code> is always smaller than input array's size for non-empty array.<br>
Answers within&nbsp;<code>10^-5</code>&nbsp;of the actual value will be accepted as correct.</p>
</div>

## Option 1

Slide the window, whilst doing a binary search on the item to add and remove

Time complexity = O(nlogk)
<br>
Space complexity = O(k)

In [ ]:
import collections
class Solution(object):
    def medianSlidingWindow(self, nums, k):
        w, half = nums[:k], k//2
        w.sort()
        res = [(float(w[half]+w[half-1]))/2 if k%2 == 0 else w[half]]
        for i in range(k,len(nums)):
            #find the index for removal, then remove form the window
            l,r = 0,k
            while l <= r:
                mid = (l+r)//2
                if w[mid] < nums[i-k]:
                    l = mid + 1
                elif w[mid] > nums[i-k]:
                    r = mid - 1
                else:
                    break
            w = w[:mid] + w[mid+1:] 
            #find the index for add, then add to the window
            l,r = 0,k-1
            while l < r:
                mid = (l+r)//2
                if w[mid] < nums[i]:
                    l = mid + 1
                else:
                    r = mid
            w = w[:l] + [nums[i]] + w[l:]
            res.append((float(w[half]+w[half-1])/2) if k%2 == 0 else w[half])           
        return res
                
Solution().medianSlidingWindow([1,3,-1,-3,5,3,6,7], 4)

#### Result: 1356ms (5.88%)

## Option 2

Remove and Add the index using inbuild bisect and insort

In [ ]:
import collections
from bisect import bisect, insort
class Solution(object):
    def medianSlidingWindow(self, nums, k):
        w, half = nums[:k], k//2
        w.sort()
        res = [(float(w[half]+w[half-1]))/2 if k%2 == 0 else w[half]]
        for i in range(k,len(nums)):
            #find the index for removal, then remove form the window
            w.pop(bisect(w,nums[i-k])-1)
            #find the index for add, then add to the window
            insort(w,nums[i])
            res.append((float(w[half]+w[half-1])/2) if k%2 == 0 else w[half])           
        return res
                
Solution().medianSlidingWindow([1,3,-1,-3,5,3,6,7], 3)

#### Result: 104ms (88.24%)

## Option 3

Min and Max heap
<li>We maintain 2 heaps, for the smallest and biggest values
<li>In order to calculate the median a balancer is needed 

In [ ]:
import collections
from heapq import heappush, heappop
class Solution(object):
    def medianSlidingWindow(self, nums, k):
        max_h, min_h, removal = [],[], collections.Counter()
        for v in nums[:k]:
            heappush(max_h,-v)
        for i in range(k/2):
            heappush(min_h,-heappop(max_h))
        res = [(float(-max_h[0]+min_h[0])/2) if k%2 == 0 else -max_h[0]]         
        for i in range(k,len(nums)):
            #Remove from window
            removal[nums[i-k]] += 1            
            balance = -1 if nums[i-k] > -max_h[0] else 1       
            #Add to window
            if nums[i] > -max_h[0] :
                heappush(min_h,nums[i])
                balance += 1
            else:
                heappush(max_h,-nums[i])
                balance -= 1 
            #Balance heaps
            if balance > 0:
                heappush(max_h, -heappop(min_h))
                balance -= 1
            elif balance < 0:
                heappush(min_h, -heappop(max_h))
                balance += 1
            #Remove the items that are in our map and appear top of the heaps
            while max_h and removal[-max_h[0]] > 0:
                removal[-heappop(max_h)] -= 1
            while min_h and removal[min_h[0]] > 0 :
                removal[heappop(min_h)] -= 1
            
            res.append((float(-max_h[0]+min_h[0])/2) if k%2 == 0 else -max_h[0])
        return res
    
        
Solution().medianSlidingWindow([1,2,3,4,2,3,1,4,2], 6)

#### Result: 164ms (42.99%)